In [2]:
# import packages
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import time
import geopy
import re
import csv
from geopy.distance import geodesic

# Extract postcodes from df and get geolocation data from OneMap API

In [5]:
# load Michelin database
df = pd.read_csv('restaurant_postcodes.csv')

In [6]:
df.head()

,Unnamed: 0,restaurant,postcode,michelin_grade
0,0,Poh Cheu,150127,The MICHELIN Plate: Good cooking
1,1,New Rong Liang Ge Cantonese Roast Duck Double ...,182269,The MICHELIN Plate: Good cooking
2,2,Chef Kang's Noodle House,319579,"Bib Gourmand: good quality, good value cooking"
3,3,Guan Kee Fried Kway Teow,270020,"Bib Gourmand: good quality, good value cooking"
4,4,Shi Le Yuan,150085,The MICHELIN Plate: Good cooking


In [53]:
df = df.dropna()
df.head()

,Unnamed: 0,0
0,Poh Cheu,150127
1,New Rong Liang Ge Cantonese Roast Duck Double ...,182269
2,Chef Kang's Noodle House,319579
3,Guan Kee Fried Kway Teow,270020
4,Shi Le Yuan,150085


### Extract postcode data

In [54]:
# find unique postcodes
postcodes = set(df['0'])

In [55]:
# check list
len(postcodes)

140

In [56]:
postcodes = list(postcodes)

In [57]:
# add postcode for homebase address, assuming we start off from Bishan Junction 8
postcodes.append('579837')

### Get postcode geolocation data from OneMap API

In [58]:
# create a dictionary to store the corresponding returns from Onemap for each postcode
store = {}

In [59]:
# for every postcode
for i in postcodes:
    # create http address
    address = 'https://developers.onemap.sg/commonapi/search?searchVal=' +i + '&returnGeom=Y&getAddrDetails=Y'
    # request for address
    request = requests.get(address)
    # prevent unsuccessful gets. sleep and try again
    while request.status_code != 200:
        time.sleep(10)
        request = requests.get(address)
    # extract request json
    request_json = request.json()
    holder = {}
    holder['found'] = request_json['found']
    # extract information
    if request_json['found'] >=1:
        holder['postal'] = request_json['results'][0]['POSTAL']
        holder['X'] = request_json['results'][0]['X']
        holder['Y'] = request_json['results'][0]['Y']
        holder['lat'] = request_json['results'][0]['LATITUDE']
        holder['long'] = request_json['results'][0]['LONGITUDE']
        holder['address'] = request_json['results'][0]['ADDRESS']
        store[i] = holder
    else:
        store[i] = holder

In [60]:
# check that all postcodes were run through
len(store)

141

In [61]:
# convert to dataframe
df_postcodes = pd.DataFrame(store).transpose()

In [62]:
df_postcodes.head(10)

,X,Y,address,found,lat,long,postal
460208,38879.2655228842,34214.8727074508,208 NEW UPPER CHANGI ROAD BEDOK TOWN CENTRE SI...,1,1.32569973464369,103.931074627428,460208
068815,29710.4517250033,28878.2697737403,6A SHENTON WAY DOWNTOWN GALLERY SINGAPORE 068815,4,1.27743927179095,103.848687622624,068815
228051,29728.9747019261,31611.3744489104,28 WILKIE ROAD SOPHIA FLATS SINGAPORE 228051,1,1.30215650857016,103.848854208182,228051
039797,30782.8866747925,30340.2235875968,"5 RAFFLES AVENUE MANDARIN ORIENTAL, SINGAPORE ...",1,1.29066059179026,103.858323976383,039797
259569,26315.8802799666,33822.8862460125,1 CLUNY ROAD SINGAPORE 259569,17,1.32215664251126,103.818185872471,259569
118543,23359.273816498,28685.8491790779,121 PASIR PANJANG ROAD SINGAPORE 118543,2,1.27569879325349,103.791619964293,118543
270020,22983.3363480661,32579.350652054,20 GHIM MOH ROAD MARKET & HAWKER CENTRE (BLK 2...,1,1.31091018371374,103.788241408454,270020
779085,26318.0685341172,42722.0810778309,54 SEMBAWANG ROAD HONG HENG MANSIONS SINGAPORE...,1,1.40263778458236,103.818205033094,779085
018956,30809.4035472879,29446.1084987707,10 BAYFRONT AVENUE MAYBANK OFFSITE@MARINABAYSA...,7,1.28257452719678,103.858562162091,018956
238897,28363.4079504274,31594.5850707535,333A ORCHARD ROAD MANDARIN GALLERY SINGAPORE 2...,2,1.30200471731963,103.836583965806,238897


In [63]:
#df_postcodes = df_postcodes.dropna(subset=['Y'])

In [64]:
df_postcodes.shape

(141, 7)

In [65]:
df_postcodes.head(20)

,X,Y,address,found,lat,long,postal
460208,38879.2655228842,34214.8727074508,208 NEW UPPER CHANGI ROAD BEDOK TOWN CENTRE SI...,1,1.32569973464369,103.931074627428,460208
068815,29710.4517250033,28878.2697737403,6A SHENTON WAY DOWNTOWN GALLERY SINGAPORE 068815,4,1.27743927179095,103.848687622624,068815
228051,29728.9747019261,31611.3744489104,28 WILKIE ROAD SOPHIA FLATS SINGAPORE 228051,1,1.30215650857016,103.848854208182,228051
039797,30782.8866747925,30340.2235875968,"5 RAFFLES AVENUE MANDARIN ORIENTAL, SINGAPORE ...",1,1.29066059179026,103.858323976383,039797
259569,26315.8802799666,33822.8862460125,1 CLUNY ROAD SINGAPORE 259569,17,1.32215664251126,103.818185872471,259569
118543,23359.273816498,28685.8491790779,121 PASIR PANJANG ROAD SINGAPORE 118543,2,1.27569879325349,103.791619964293,118543
270020,22983.3363480661,32579.350652054,20 GHIM MOH ROAD MARKET & HAWKER CENTRE (BLK 2...,1,1.31091018371374,103.788241408454,270020
779085,26318.0685341172,42722.0810778309,54 SEMBAWANG ROAD HONG HENG MANSIONS SINGAPORE...,1,1.40263778458236,103.818205033094,779085
018956,30809.4035472879,29446.1084987707,10 BAYFRONT AVENUE MAYBANK OFFSITE@MARINABAYSA...,7,1.28257452719678,103.858562162091,018956
238897,28363.4079504274,31594.5850707535,333A ORCHARD ROAD MANDARIN GALLERY SINGAPORE 2...,2,1.30200471731963,103.836583965806,238897


In [66]:
# write to csv
df_postcodes.to_csv('Michelin_geolocations.csv')